In [2]:
# data_files = "alpaca_data.json"
# data_files = "t2c_concode/t2c_answers.json"
data_files = "/root/data/t2c_train.json"#"t2c_concode/t2c_1000train.json"
# data_files = "t2c_concode/t2c_10000train.json"

# data_files = "t2c_concode/t2c_train.json"
experiment_name = "/root/experiments/t2c_concode_220428_v12"

default_model = "decapoda-research/llama-7b-hf"

# Setting for A100 - For 3090 
MICRO_BATCH_SIZE = 2#4 # 8  # change to 4 for 3090
BATCH_SIZE = 10#32#128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 10#20  # paper uses 3
LEARNING_RATE = 2e-3  # from the original paper
CUTOFF_LEN = 384 # 256 accounts for about 96% of the data
LORA_R = 4*2
LORA_ALPHA = 16#*2
LORA_DROPOUT = 0.05

# Trainer config
warmup_steps=2000
fp16=True
logging_steps=10
save_total_limit=1
save_strategy = 'steps'
save_steps = 50
seed=42
logging_strategy='steps'
report_to='tensorboard'
mlm = False

truncation=True
padding="max_length"

config_use_cache = False
resume_from_checkpoint = True

In [3]:
first_time = False

In [4]:
if first_time:
    !git clone https://github.com/tloen/alpaca-lora.git

In [5]:
!nvidia-smi

# collab parametres (cuda version should be the same)
# Sat Apr 22 18:27:40 2023       
# +-----------------------------------------------------------------------------+
# | NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
# |-------------------------------+----------------------+----------------------+
# | GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
# | Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
# |                               |                      |               MIG M. |
# |===============================+======================+======================|
# |   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
# | N/A   32C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
# |                               |                      |                  N/A |
# +-------------------------------+----------------------+----------------------+
                                                                               
# +-----------------------------------------------------------------------------+
# | Processes:                                                                  |
# |  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
# |        ID   ID                                                   Usage      |
# |=============================================================================|
# |  No running processes found                                                 |
# +-----------------------------------------------------------------------------+

Wed May  3 10:03:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:1A:00.0 Off |                  N/A |
|  0%   50C    P8    16W / 300W |      1MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Alpaca LLaMa 7B LoRa

In [6]:
# !pip install datasets loralib sentencepiece
# # is it okay version?
# !pip install transformers
# # is it okay version?
# !pip install peft
# !pip install bitsandbytes

In [7]:
if first_time:
    !pip install datasets loralib sentencepiece

    !pip uninstall transformers
    !pip install git+https://github.com/zphang/transformers@c3dc391
    # pip install git+https://github.com/zphang/transformers.git@llama_push

    !pip install git+https://github.com/huggingface/peft.git

In [8]:
if first_time:
    !pip install bitsandbytes

## Fine Tuning

In [9]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig#, LLaMAForCausalLM, LLaMATokenizer
from transformers import LlamaForCausalLM as LLaMAForCausalLM
from transformers import LlamaTokenizer as LLaMATokenizer

from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/usr/local/cuda/lib64/libcudart.so'), PosixPath('/usr/local/cuda/lib64/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [10]:
# # Setting for A100 - For 3090 
# MICRO_BATCH_SIZE = 2#4 # 8  # change to 4 for 3090
# BATCH_SIZE = 32#128
# GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
# EPOCHS = 20  # paper uses 3
# LEARNING_RATE = 2e-3  # from the original paper
# CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
# LORA_R = 4
# LORA_ALPHA = 16
# LORA_DROPOUT = 0.05

In [11]:
model = LLaMAForCausalLM.from_pretrained(
    default_model,
    load_in_8bit=True,
    device_map="auto",
)
tokenizer = LLaMATokenizer.from_pretrained(
    default_model, add_eos_token=True
)

model = prepare_model_for_int8_training(model)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [12]:
# # data_files = "alpaca_data.json"
# # data_files = "t2c_concode/t2c_answers.json"
# data_files = "t2c_concode/t2c_1000train.json"
# # data_files = "t2c_concode/t2c_10000train.json"

# # data_files = "t2c_concode/t2c_train.json"
# experiment_name = "t2c_concode_220428_v5"

In [13]:
import json
import numpy as np
from collections import Counter

lst_temp = json.load(open(data_files, 
                          'r'
                         )
                    )

instruct_keys = tuple(['instruction', 'input', 'output'])
assert set([tuple(i.keys()) for i in lst_temp]), set([instruct_keys])
print("all", len(lst_temp))
for key in instruct_keys:
    lst = [i[key] for i in lst_temp]
    n_list = np.array([len(i) for i in lst])
    print("key=%s min=%d max=%d mean=%3.3f median=%3.3f q97=%3.3f"%(key, 
                                                                    max(n_list), 
                                                                    min(n_list), 
                                                                    n_list.mean(), 
                                                                    np.median(n_list), 
                                                                    np.quantile(n_list, 0.95)
                                                                   )
         )
# # instruction

all 100000
key=instruction min=2878 max=21 mean=102.040 median=72.000 q97=257.000
key=input min=4419 max=20 mean=461.623 median=360.000 q97=1249.000
key=output min=663 max=20 mean=117.702 median=85.000 q97=293.000


In [14]:
# sorted(Counter([i['input'] for i in lst_temp]).items())[:10]

In [15]:
# # model
# https://stackoverflow.com/questions/75969294/assertionerror-no-inf-checks-were-recorded-for-this-optimizer
# model = get_peft_model(model, config)
# model = PeftModel.from_pretrained(model, LORA_WEIGHTS, torch_dtype=torch.float16)

In [ ]:
from peft import PeftModel

def load_model_and_tokenizer(experiment_name = "/root/experiments/t2c_concode_220428_v12",
                             LORA_R = 4*2,
                             LORA_ALPHA = 16,#*2
                             LORA_DROPOUT = 0.05
                            ):


    """
        
    """
    
    config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = get_peft_model(model, config)

    tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
    data = load_dataset("json", 
                        data_files = data_files
                       )
    return model, tokenizer

In [16]:
from peft import PeftModel
# model = PeftModel.from_pretrained(model, experiment_name, torch_dtype=torch.float16)

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
data = load_dataset("json", 
                    data_files = data_files
                   )

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-7514164041f99238/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [79]:
!ls {experiment_name}

1683108328.8478951   checkpoint-20500
adapter_config.json  events.out.tfevents.1683108328.8d048d63ed1a.4412.0
adapter_model.bin


In [84]:
!head {experiment_name}/adapter_config.json

{
  "base_model_name_or_path": "decapoda-research/llama-7b-hf",
  "bias": "none",
  "enable_lora": null,
  "fan_in_fan_out": false,
  "inference_mode": true,
  "lora_alpha": 16,
  "lora_dropout": 0.05,
  "merge_weights": false,
  "modules_to_save": null,


In [85]:
!cp {experiment_name}/adapter_config.json {experiment_name}/config.json 

In [88]:
# model.config.to_json()
!ls {experiment_name+'/'+"checkpoint-20500/"}

optimizer.pt	   rng_state.pth  scheduler.pt	      training_args.bin
pytorch_model.bin  scaler.pt	  trainer_state.json


In [103]:
model.print_trainable_parameters()

trainable params: 4194304 || all params: 6742609920 || trainable%: 0.06220594176090199


In [100]:
# config = model.config
f"{experiment_name}/config.json"

'/root/experiments/t2c_concode_220428_v12/config.json'

In [99]:
model.from_pretrained(experiment_name+'/'+"checkpoint-20500/pytorch_model.bin", f"{experiment_name}/config.json")

ValueError: Can't find config.json at '/root/experiments/t2c_concode_220428_v12/config.json'

In [17]:
print("data_files: ", data_files)

data_files:  /root/data/t2c_train.json


In [ ]:
# experiment_name+"_v2"
# vim /opt/conda/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py
# !df -h .
resume_from_checkpoint

In [ ]:
def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}
### Response:
{data_point["output"]}"""


data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=truncation,
        max_length=CUTOFF_LEN,
        padding=padding
    )
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=warmup_steps,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=fp16,
        logging_steps=logging_steps,
        output_dir=experiment_name,#"lora-alpaca",
        save_total_limit=save_total_limit,
        save_strategy = save_strategy,
        save_steps = save_steps,
        seed=seed,
        logging_dir=experiment_name,
        logging_strategy=logging_strategy,
        report_to=report_to
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=mlm),
)
model.config.use_cache = config_use_cache
# print(len(trainer.optimizer.state['found_inf_per_device']))
trainer.train(resume_from_checkpoint=resume_from_checkpoint)


model.save_pretrained(experiment_name)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,2.462900
20,2.476200
30,2.496400
40,2.365400
50,2.228300
60,1.898900
70,1.632900
80,1.339100
90,1.204200
100,1.121300


In [22]:
model.save_pretrained(experiment_name)

In [1]:
model.eval();

NameError: name 'model' is not defined

In [24]:
assert model.training == False

In [25]:
import json
import numpy as np
import math
import tqdm
from transformers import GenerationConfig


# def generate_prompt(instruction, input):
#     return f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"

def generate_test_prompt(data_point, train = False):
    # To decrease expectations of results :)
    assert train == False
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"] if train else ''}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"] if train else ''}"""


lst = json.load(open('../data/t2c_answers.json', 'rb'))
inputs = lst# [lst[0]]
# instruction = 'Combine the question and answer into an image caption as succinctly as possible. Be sure to include the phrase "a photo of". Do not draw false conclusions.'
# inputs = ['Is this a baseball game? yes', 'Is this a baseball game? no']
prompts = [generate_test_prompt(inp) for inp in inputs]
prompts = np.array(prompts)
batch_size = 9
res_list = []
for ind in tqdm.tqdm(range(math.ceil(len(prompts)/batch_size))):
    current_prompts = prompts[ind*batch_size: (ind+1)*batch_size]
    print(ind*batch_size, (ind+1)*batch_size, len(current_prompts))
    
    tokenized_inputs = tokenizer(list(current_prompts), 
                                 padding=True, 
                                 truncation=True, 
                                 return_tensors="pt"
                                ).to('cuda')

    generation_config = GenerationConfig(max_new_tokens=128)

    with torch.no_grad():
        full_output = model.generate(
            **tokenized_inputs,
            generation_config=generation_config
        )

    res_list.extend(tokenizer.batch_decode(full_output, skip_special_tokens=False))
#     print(tokenizer.batch_decode(full_output, skip_special_tokens=False)[-1])


    


  0%|                                                    | 0/12 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0 9 9


  8%|███▋                                        | 1/12 [00:20<03:43, 20.29s/it]

9 18 9


 17%|███████▎                                    | 2/12 [00:40<03:20, 20.07s/it]

...

 83%|███████████████████████████████████▊       | 10/12 [03:19<00:39, 19.74s/it]

90 99 9


 92%|███████████████████████████████████████▍   | 11/12 [03:39<00:19, 19.71s/it]

99 108 1


100%|███████████████████████████████████████████| 12/12 [03:55<00:00, 19.61s/it]


In [47]:
from datetime import datetime
import json
import numpy as np
import math
import tqdm
from transformers import GenerationConfig

from T2CEvaluator import T2CEvaluator


    
def generate_test_prompt(data_point, train = False):
    # To decrease expectations of results :)
    assert train == False
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"] if train else ''}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"] if train else ''}"""


class EvaluateTestSet:
    def __init__(self, 
                 generation_config = GenerationConfig(max_new_tokens=128), 
                 fn_test_data = "../data/t2c_answers.json",
                 fn_etalon = "/root/data/answers.json",
                 batch_size = 10
                ):
        self.generation_config = generation_config
        
        self.fn_test_data = fn_test_data
        self.fn_etalon = fn_etalon
        
        self.batch_size = batch_size
    
    @classmethod
    def preprocess(cls, s, verbose = False):
        #ToDo rewrite it using Promt Template
        s = s.split('### Response:\n')[-1]
        s = s.replace('\n', '  ')
        s = s.replace('<unk>', " ")
        s = ' '.join(s.split(' ')[:100])
        while '  ' in s:
            s = s.replace('  ', ' ')

        if len(s) > 0 and s[0] == ' ':
            s = s[1:]
        
        if verbose:
            print(s)
        
        return s

    def clean_results(self, res_list):
        predict_list = []
        for s in tqdm.tqdm(res_list):
            predict_list.append(self.preprocess(s))
        return predict_list
    
    def get_raw_results(self, model, prompts):
        batch_size = self.batch_size
        generation_config = self.generation_config
        
        res_list = []
        n = math.ceil(len(prompts)/batch_size)
        
        for ind in tqdm.tqdm(range(n)):
            current_prompts = prompts[ind*batch_size: (ind+1)*batch_size]
            print(ind*batch_size, (ind+1)*batch_size, len(current_prompts))

            tokenized_inputs = tokenizer(list(current_prompts), 
                                         padding=True, 
                                         truncation=True, 
                                         return_tensors="pt"
                                        ).to('cuda')



            with torch.no_grad():
                full_output = model.generate(
                    **tokenized_inputs,
                    generation_config=generation_config
                )

            res_list.extend(tokenizer.batch_decode(full_output, skip_special_tokens=False))
        
        return res_list
    
    def save_results(self, predict_list):
        output_filename = str(datetime.now()).split('.')[0].replace(' ', '-').replace(':', '_')+'.txt'
        fn_output = "/root/results/%s"%output_filename
        
        res = '\n'.join([i if i!='' else '-' for i in predict_list])
        open(fn_output, "w+", encoding='utf-8').write(res)
        return fn_output
    
    def evaluate(self, model):
        model.eval()
        assert model.training == False

        lst = json.load(open(self.fn_test_data, 'rb'))
        inputs = lst# [lst[0]]
        # instruction = 'Combine the question and answer into an image caption as succinctly as possible. Be sure to include the phrase "a photo of". Do not draw false conclusions.'
        # inputs = ['Is this a baseball game? yes', 'Is this a baseball game? no']
        prompts = [generate_test_prompt(inp) for inp in inputs]
        prompts = np.array(prompts)
        
        res_list = self.get_raw_results(model = model, 
                                        prompts = prompts)
        
        model.train()
        assert model.training == True
        
        predict_list = self.clean_results(res_list)
        
        self.fn_output = self.save_results(predict_list)
        
        t2c_evaluator = T2CEvaluator()
        metric_res = t2c_evaluator.calculate_metrics(fn_answers = self.fn_etalon, 
                                                     fn_predictions = self.fn_output
                                                     )
        return metric_res

In [ ]:
# load model


{'EM': 0.0,
 'BLEU': 0.09414554673739628,
 'brevity_penalty': 1.0,
 'ratio': 1.0386100386100385,
 'translation_length': 3228,
 'reference_length': 3108,
 'precisions_0': 0.2003716320842366,
 'precisions_1': 0.12304250559284116,
 'precisions_2': 0.07168342523516055,
 'precisions_3': 0.04445176160684886}

In [53]:
from EvaluateTestSet2 import EvaluateTestSet

In [54]:
evaluator = EvaluateTestSet()
metric_res = evaluator.evaluate(model=model, tokenizer=tokenizer)
metric_res

  0%|                                                    | 0/10 [00:00<?, ?it/s]

0 10 10


 10%|████▍                                       | 1/10 [00:20<03:04, 20.51s/it]

10 20 10


 20%|████████▊                                   | 2/10 [00:40<02:39, 19.99s/it]

...

80 90 10


 90%|███████████████████████████████████████▌    | 9/10 [02:56<00:19, 19.26s/it]

90 100 10


100%|██████████████████████████████████████| 100/100 [00:00<00:00, 79800.30it/s]


UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 3952: ordinal not in range(128)

In [77]:
# type(model)
# model.T_destination
# model.bfloat16

In [72]:
model.__dir__()

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_create_repo',
 '_forward_hooks',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_files_timestamps',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register

In [67]:
t2c_evaluator = T2CEvaluator()
t2c_evaluator.calculate_metrics(fn_answers = evaluator.fn_etalon,
                                fn_predictions = evaluator.fn_output
                               )

{'EM': 0.0,
 'BLEU': 0.09843311907126237,
 'brevity_penalty': 1.0,
 'ratio': 1.0431145431145432,
 'translation_length': 3242,
 'reference_length': 3108,
 'precisions_0': 0.19272278754239902,
 'precisions_1': 0.12599427298759147,
 'precisions_2': 0.07714654615881214,
 'precisions_3': 0.05011464133639044}

In [70]:
# from T2CEvaluator import T2CEvaluator

In [69]:
# evaluator.fn_output

In [68]:
# open("/root/results/2023-05-04-18_20_22.txt", "r", encoding='utf-8').readlines()

'/root/results/2023-05-04-18_20_22.txt'

In [71]:
import subprocess
# command = f"python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a /root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json -p {current_pred_name}"
command = f"python /root/code/CodeXGLUE_t2c_evaluator/evaluator.py -a {evaluator.fn_etalon} -p {evaluator.fn_output}"

result = subprocess.run(command.split(' '), 
                        stderr=subprocess.PIPE)
result.stderr
results = result.stderr.decode()
print(results)

INFO:__main__:BLEU: 9.84, EM: 0.0



In [46]:
import subprocess
# command = f"python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a /root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json -p {current_pred_name}"
command = f"python /root/code/CodeXGLUE_t2c_evaluator/evaluator.py -a {evaluator.fn_etalon} -p {evaluator.fn_output}"

result = subprocess.run(command.split(' '), 
                        stderr=subprocess.PIPE)
result.stderr
results = result.stderr.decode()
print(results)

INFO:__main__:BLEU: 9.41, EM: 0.0



In [26]:
def preprocess(s):
    s = s.split('### Response:\n')[-1]
    s = s.replace('\n', '  ')
    s = s.replace('<unk>', " ")
    s = ' '.join(s.split(' ')[:100])
    while '  ' in s:
        s = s.replace('  ', ' ')
    
    if len(s) > 0 and s[0] == ' ':
        s = s[1:]
    print(s)
    return s


predict_list = []
for s in tqdm.tqdm(res_list):
    predict_list.append(preprocess(s))

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 22797.61it/s]


if ( fStatementsRecovery ) { fRewrite = fRewrite . getRewrite ( ) ; fImportRewrite = fImportRewrite . getRewrite ( ) ; fImportRemover = fImportRemover . getRewrite ( ) ; fResolveBindings = fResolveBindings . getRewrite ( ) ; } } return fStatementsRecovery ; } return false ; } } return fRewrite . getRewrite ( ) . getStatementsRecovery ( ) ; } } return false ; } } return fRewrite . getRewrite ( ) . getStatements

String function ( ) { return baseZNode ; } } catch ( Exception loc0 ) { return null ; } } } return null ; } } } } } } } } return null ; } } } } }
boolean function ( C arg0 , C arg1 ) { return arg0 . compareTo ( arg1 ) ; } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } }

FunctionResult function ( UnivariateRealFunction arg0 , double arg1 , double arg2 ) { return getSolver ( arg0 , arg1 , arg2 ) ; } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } }






BaseColor function ( ) { return strokeColor . getColorSpa

In [27]:
# predict_list

In [28]:
!mkdir /root/results

mkdir: cannot create directory ‘/root/results’: File exists


In [29]:
from datetime import datetime
output_filename = str(datetime.now()).split('.')[0].replace(' ', '-').replace(':', '_')+'.txt'
current_pred_name = "/root/results/%s"%output_filename
print(current_pred_name)

/root/results/2023-05-04-12_53_56.txt


In [36]:
from T2CEvaluator import T2CEvaluator
T2CEvaluator().calculate_metrics(fn_answers = "/root/data/answers.json", 
                                 fn_predictions = current_pred_name
                                )

{'EM': 0.0,
 'BLEU': 0.09414554673739628,
 'brevity_penalty': 1.0,
 'ratio': 1.0386100386100385,
 'translation_length': 3228,
 'reference_length': 3108,
 'precisions_0': 0.2003716320842366,
 'precisions_1': 0.12304250559284116,
 'precisions_2': 0.07168342523516055,
 'precisions_3': 0.04445176160684886}

In [30]:
res = '\n'.join([i if i!='' else '-' for i in predict_list])
open(current_pred_name, "w+", encoding='utf-8').write(res)
# 8724
# 7124
# 9000

10529

In [33]:
res.count('\n') 
# 99

99

In [34]:
current_pred_name

'/root/results/2023-05-04-12_53_56.txt'

In [32]:
import subprocess
# command = f"python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a /root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json -p {current_pred_name}"
command = f"python /root/code/CodeXGLUE_t2c_evaluator/evaluator.py -a /root/data/answers.json -p {current_pred_name}"

result = subprocess.run(command.split(' '), 
                        stderr=subprocess.PIPE)
result.stderr
results = result.stderr.decode()
print(results)

# 1 hour INFO:__main__:BLEU: 9.56, EM: 0.0
# 100 examples (loss 0.1, ~10 hours) INFO:__main__:BLEU: 11.22, EM: 0.0
# 1000 examples x 15 hours (~15 epoch) INFO:__main__:BLEU: 11.13, EM: 0.0
# lrX(1e-1) => INFO:__main__:BLEU: 6.82, EM: 0.0
# 20 min INFO:__main__:BLEU: 6.13, EM: 0.0

# 1 day, all train 1.7 times: INFO:__main__:BLEU: 7.9, EM: 0.0
# 15 min params*2: INFO:__main__:BLEU: 5.27, EM: 0.0

# 12 hours data x 1.2 params*2: INFO:__main__:BLEU: 6.38, EM: 0.0
# INFO:__main__:BLEU: 9.41, EM: 0.0

INFO:__main__:BLEU: 9.41, EM: 0.0



In [50]:
import subprocess
# command = f"python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a /root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json -p {current_pred_name}"
command = f"python /root/code/CodeXGLUE_t2c_evaluator/evaluator.py -a /root/data/answers.json -p {current_pred_name}"

result = subprocess.run(command.split(' '), 
                        stderr=subprocess.PIPE)
result.stderr
results = result.stderr.decode()
print(results)

# 1 hour INFO:__main__:BLEU: 9.56, EM: 0.0

INFO:__main__:BLEU: 11.22, EM: 0.0



In [19]:
model.training

True

In [42]:
model = model.eval()

In [46]:
# model = model()

In [39]:
experiment_name

't2c_concode_220428_v4'

In [43]:
m = model.to('cpu')

In [45]:
m.save_pretrained(experiment_name)

OutOfMemoryError: CUDA out of memory. Tried to allocate 44.00 MiB (GPU 0; 23.70 GiB total capacity; 20.82 GiB already allocated; 14.81 MiB free; 22.62 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [32]:
import gc
import GPUtil
import torch
GPUtil.showUtilization()

ModuleNotFoundError: No module named 'GPUtil'

In [33]:
!df -h .

Filesystem      Size  Used Avail Use% Mounted on
overlay          45G   24G   22G  54% /


In [23]:
# model = model.to("cpu")
model.to("cpu")

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): LLaMAForCausalLM(
          (model): LLaMAModel(
            (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
            (layers): ModuleList(
              (0-31): 32 x LLaMADecoderLayer(
                (self_attn): LLaMAAttention(
                  (q_proj): Linear8bitLt(
                    in_features=4096, out_features=4096, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=4096, out_features=4, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=4, out_features=4096, bias=False)
                    )
                  )
                  (k_proj): Linear8bitLt(in_features=4096, o

In [25]:
m = model.to('cpu')

In [30]:
model.__dir__()

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_create_repo',
 '_forward_hooks',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_files_timestamps',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register

In [35]:
experiment_name

't2c_concode_220428_v6'

In [32]:
!ls {experiment_name}

1682786829.6937962  events.out.tfevents.1682786829.d0bb1c4a2cfe.3455.0
1682787745.9152367  events.out.tfevents.1682787745.d0bb1c4a2cfe.3590.0


In [33]:
model.save_pretrained(experiment_name)
# model.save_pretrained()

In [34]:
!ls {experiment_name}

1682786829.6937962   adapter_model.bin
1682787745.9152367   events.out.tfevents.1682786829.d0bb1c4a2cfe.3455.0
adapter_config.json  events.out.tfevents.1682787745.d0bb1c4a2cfe.3590.0


In [33]:
experiment_name+"_v3"

't2c_concode_220428_v3'

In [ ]:
!ls alpaca_finetuned_gpu

ls: cannot access 'alpaca_finetuned_gpu': No such file or directory


In [66]:
!ls lora-alpaca_finetuned

adapter_config.json  adapter_model.bin


In [72]:
model.device

device(type='cuda', index=0)

In [21]:
!df -h .

Filesystem      Size  Used Avail Use% Mounted on
overlay          45G   17G   29G  37% /


In [51]:
model.to('cuda')

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LLaMAForCausalLM(
      (model): LLaMAModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LLaMADecoderLayer(
            (self_attn): LLaMAAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
              )
              (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
          

In [25]:
# !apt install zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  zip
0 upgraded, 1 newly installed, 0 to remove and 43 not upgraded.
Need to get 167 kB of archives.
After this operation, 638 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 zip amd64 3.0-11build1 [167 kB]
Fetched 167 kB in 5s (34.8 kB/s)
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package zip.
(Reading database ... 16793 files and directories currently installed.)
Preparing to unpack .../zip_3.0-11build1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 17%] [#########.................................................] 8Unpacking zip (3.0-11build1) ...
7Progress: [ 33%] [###################.......................................] 87Progress: [ 50%] [################

In [98]:
!zip -r lora-alpaca.zip lora-alpaca

  adding: lora-alpaca/ (stored 0%)
  adding: lora-alpaca/checkpoint-500/ (stored 0%)
  adding: lora-alpaca/checkpoint-500/pytorch_model.bin
zip I/O error: No space left on device
zip error: Output file write failure (write error on zip file)


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("samwit/alpaca7B-lora", use_auth_token=True)

## Generation

In [ ]:
%cd /root/alpaca-lora

In [19]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

generation_config = GenerationConfig(
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
    max_length = 1068,
)

In [2]:
!ls ..

NGC-DL-CONTAINER-LICENSE  dev	lib    mnt   root  srv	usr
bin			  etc	lib64  opt   run   sys	var
boot			  home	media  proc  sbin  tmp	workspace


In [3]:
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
tokenizer = LLaMATokenizer.from_pretrained(
    "decapoda-research/llama-7b-hf", add_eos_token=True
)


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [4]:
# tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")

# model = LLaMAForCausalLM.from_pretrained(
#     "decapoda-research/llama-7b-hf",
#     load_in_8bit=True,
#     device_map="auto",
# )

model = PeftModel.from_pretrained(model, experiment_name)#"lora-alpaca_finetuned_gpu")

ValueError: Can't find 'adapter_config.json' at 'lora-alpaca_finetuned_gpu'

In [22]:
# generation_config = GenerationConfig(
#     max_length = 1068,
#     min_length=1068,
#     temperature=0.6,
#     top_p=0.9,
#     repetition_penalty=1.15,
#     early_stopping = False,
#     max_time = 60,
#     num_beams = 20
# )


PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Tell me something about alpacas.
### Response:"""

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()


print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    min_length=506
#     max_new_tokens=1024
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Tell me something about alpacas.
### Response:


In [116]:
# model

In [21]:
tokenizer.decode(generation_output.sequences[0])

' Below is an instruction that describes a task. Write a response that appropriately completes the request.\n### Instruction:\nTell me something about alpacas.\n### Response:'

In [23]:
PROMPT ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an ode to why do Alpacas make the best pets?

### Response:
'''

In [128]:
# https://github.com/tloen/alpaca-lora/issues/404
# model.config.pad_token_id = tokenizer.pad_token_id = 0 # unk
# model.config.bos_token_id = 1
# model.config.eos_token_id = 2

In [127]:
PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
 Tell me something about alpacas.
### Response:"""
PROMPT = PROMPT.replace('\n', ' ')
# %%time

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

generation_config = GenerationConfig(
    do_sample=True,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.15,
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=True,
    min_new_tokens=128,
    max_length=1000,
    min_length=500
)
for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction:  Tell me something about alpacas. ### Response:


In [117]:
input_ids

tensor([[    1, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29889,
         14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
         29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13,  6113,
           385,   288,   311,   304,  2020,   437,   838, 29886,   562,   294,
          1207,   278,  1900,   282,  1691, 29973,    13,    13,  2277, 29937,
         13291, 29901,    13,     2]], device='cuda:0')

In [25]:
!pwd 

/root/alpaca-lora


In [26]:
#old
# import json
# def load_jsonl(fn):
#     """
#         Read jsonl file
#     """
#     return [json.loads(i) for i in open(fn, 'r').read().split('\n') if i!= '']


# def get_prompt(ind):
#     # concode_field_sep
#     text = dct[ind]['nl'].replace('concode_field_sep', '\nsignatures:')
#     PROMPT =f'''Below is an instruction that describes a task. Write a response that appropriately completes the request.
#                ### Instruction:
#                Generate the java method code
#                {text}
#                ### Response:'''
#     return PROMPT

# # dct = load_jsonl("/root/data/CodeXGLUE/Text-Code/text-to-code/dataset/concode/train.json")
# dct = load_jsonl("/root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json")
# print(len(dct))

In [27]:
import json
def generate_prompt(data_point, train = False):
    # sorry about the formatting disaster gotta move fast
    if data_point["input"]:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}

### Input:
{data_point["input"]}

### Response:
{data_point["output"] if train else ''}"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{data_point["instruction"]}

### Response:
{data_point["output"] if train else ''}"""

dct = json.load(open('t2c_concode/t2c_answers.json', "r"))

def get_prompt(ind, dct = dct):
    res = generate_prompt(dct[ind])
    res = res.split('### Response:')[0] + "### Response:"
    return res



In [97]:
def inference(PROMPT):
    inputs = tokenizer(
        PROMPT,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
#         temperature=0.6,#1-0.001,
#         top_p=0.7,
#         repetition_penalty=1.15,
        renormalize_logits = True,
        min_new_tokens = 1000,
        max_time = 50,
        use_cache = False
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=1280,
    )
    res = ""
    for s in generation_output.sequences:
#         res += tokenizer.decode(s) + '\n'
        print(tokenizer.decode(s))
    return generation_output

In [98]:
prompt = generate_prompt({"instruction": "test",
                 "input": "test",
                 "output": "test"
                }
               )
prompt = generate_prompt(data['train'][0])
prompt = prompt.split('### Response:')[0]#+'\n### Response:'
generation_output = inference(prompt)

Generating...
 Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Generate java code
evaluates if the provided holiday instance is valid for the provided year. 

### Input:
 String THREE_YEARS 
 String EVERY_YEAR 
 String ODD_YEARS 
 String TWO_YEARS 
 String FIVE_YEARS 
 String FOUR_YEARS 
 String EVEN_YEARS 
 XMLUtil xmlUtil 
 String SIX_YEARS 
 CalendarUtil calendarUtil 

 LocalDate moveDate 
 LocalDate moveDate 
 boolean isValidInYear 
 LocalDate getEasterSunday 
 boolean shallBeMoved 
 boolean isValidForCycle




In [102]:
tokenizer.decode(generation_output['sequences'][0])

' Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\nGenerate java code\nevaluates if the provided holiday instance is valid for the provided year. \n\n### Input:\n String THREE_YEARS \n String EVERY_YEAR \n String ODD_YEARS \n String TWO_YEARS \n String FIVE_YEARS \n String FOUR_YEARS \n String EVEN_YEARS \n XMLUtil xmlUtil \n String SIX_YEARS \n CalendarUtil calendarUtil \n\n LocalDate moveDate \n LocalDate moveDate \n boolean isValidInYear \n LocalDate getEasterSunday \n boolean shallBeMoved \n boolean isValidForCycle\n\n'

In [113]:
tokenizer.__dir__()

['sp_model_kwargs',
 'init_inputs',
 'init_kwargs',
 'name_or_path',
 '_processor_class',
 'model_max_length',
 'padding_side',
 'truncation_side',
 'model_input_names',
 'deprecation_warnings',
 '_in_target_context_manager',
 '_bos_token',
 '_eos_token',
 '_unk_token',
 '_sep_token',
 '_pad_token',
 '_cls_token',
 '_mask_token',
 '_pad_token_type_id',
 '_additional_special_tokens',
 'verbose',
 'added_tokens_encoder',
 'added_tokens_decoder',
 'unique_no_split_tokens',
 'tokens_trie',
 '_decode_use_source_tokenizer',
 'vocab_file',
 'add_bos_token',
 'add_eos_token',
 'decode_with_prefix_space',
 'sp_model',
 '_no_prefix_space_tokens',
 '__module__',
 '__doc__',
 'vocab_files_names',
 'pretrained_vocab_files_map',
 '__init__',
 'no_prefix_space_tokens',
 'vocab_size',
 'bos_token_id',
 'eos_token_id',
 'get_vocab',
 '_tokenize',
 '_convert_token_to_id',
 '_convert_id_to_token',
 '_maybe_add_prefix_space',
 'convert_tokens_to_string',
 'save_vocabulary',
 'build_inputs_with_special_tok

In [114]:
tokenizer.max_model_input_sizes

{}

In [112]:
tokenizer("aab")

{'input_ids': [1, 263, 370, 2], 'attention_mask': [1, 1, 1, 1]}

In [55]:
inference("""Instruction: 
Input
Response""")

Generating...


' \n'

In [29]:
# print(dct[1]['code'])

In [30]:
# print(dct[1]['nl'])

In [31]:
# inference(dct[1]['nl'])

In [32]:
from tqdm import tqdm
def preprocess(s):
    s = s.split('### Response:\n')[-1]
    s = s.replace('\n', '  ')
    while '  ' in s:
        s = s.replace('  ', ' ')
    
    if len(s) > 0 and s[0] == ' ':
        s = s[1:]
    print(s)
    return s


res_list = []
predict_list = []
for ind in tqdm(range(len(dct))):
    PROMPT = get_prompt(ind)
    txt = inference(PROMPT)
    res_list.append(txt)
    predict_list.append(preprocess(txt))

  0%|                                                   | 0/100 [00:00<?, ?it/s]

Generating...


  1%|▍                                          | 1/100 [00:00<00:39,  2.52it/s]


Generating...


  2%|▊                                          | 2/100 [00:00<00:47,  2.06it/s]

...

 97%|████████████████████████████████████████▋ | 97/100 [00:32<00:01,  2.97it/s]


Generating...

Generating...


 99%|█████████████████████████████████████████▌| 99/100 [00:32<00:00,  3.84it/s]


Generating...

Generating...


100%|█████████████████████████████████████████| 100/100 [00:33<00:00,  3.03it/s]

In [33]:
!mkdir t2c_concode/results

mkdir: cannot create directory ‘t2c_concode/results’: File exists


In [34]:
from datetime import datetime
output_filename = str(datetime.now()).split('.')[0].replace(' ', '-').replace(':', '_')+'.txt'
current_pred_name = "/root/alpaca-lora/t2c_concode/results/%s"%output_filename
print(current_pred_name)

/root/alpaca-lora/t2c_concode/results/2023-04-28-19_29_39.txt


In [35]:
# !pwd
len(predict_list)

100

In [49]:
with open(current_pred_name, 'w+') as f:
    f.write('\n'.join(['-' for i in predict_list]))
    f.write('\n')

In [50]:
# output_file_name = "t2c_concode/res_{my_predictions}.txt"
!ls {current_pred_name}

/root/alpaca-lora/t2c_concode/results/2023-04-28-19_29_39.txt


In [51]:
open(current_pred_name, "r").read().count('\n')

100

In [52]:
open("/root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json", "r").read().count('\n')

100

In [53]:
!python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a=/root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json -p={current_pred_name}


INFO:__main__:BLEU: 0.0, EM: 0.0


In [64]:
!python /root/CodeXGLUE/Text-Code/text-to-code/evaluator/evaluator.py -a=/root/CodeXGLUE/Text-Code/text-to-code/evaluator/answers.json -p=/root/alpaca-lora/t2c_concode/results/2023-04-28-17_28_43.txt



INFO:__main__:BLEU: 72.06, EM: 29.0


In [85]:
txt = open("/root/data/CodeXGLUE/Text-Code/text-to-code/evaluator/predictions.txt", 'r').read()

In [87]:
txt.count('\n')

100

In [81]:
predict_list[0].replace('\n', ' ')

'               public void walk(int start, int end) {\n                   if (start < end){\n                       for (int i = start; i <= end; i++) {\n                           if (_bag.containsKey(i)) {\n                               _bag.get(_bag.keySet().toArray()[i]);\n                            } else {\n                                _bag.put(i);\n                            }\n                        }\n                    }\n              }\n'

In [ ]:
txt.split('### Response:')[-1]

In [71]:
PROMPT = get_prompt(1)
print(PROMPT)
inference(PROMPT)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
               ### Instruction:
               Generate the java method code
               controls whether the compiler should perform statements recovery . to be effective , this method must be called before any of #getroot , #getastrewrite , #getimportremover . this method has no effect if the target object has been created with #compilationunitrewrite icompilationunit , compilationunit . defaults to true do perform statements recovery . 

 ImportRewrite fImportRewrite concode_elem_sep ImportRemover fImportRemover concode_elem_sep WorkingCopyOwner fOwner concode_elem_sep ASTRewrite fRewrite concode_elem_sep boolean fStatementsRecovery concode_elem_sep List<TextEditGroup> fTextEditGroups concode_elem_sep ICompilationUnit fCu concode_elem_sep IDocument fRememberContent concode_elem_sep CompilationUnit fRoot concode_elem_sep boolean fResolveBindings concode_elem_sep boolean fBindi

' Below is an instruction that describes a task. Write a response that appropriately completes the request.\n               ### Instruction:\n               Generate the java method code\n               controls whether the compiler should perform statements recovery. to be effective, this method must be called before any of #getroot, #getastrewrite, #getimportremover. this method has no effect if the target object has been created with #compilationunitrewrite icompilationunit, compilationunit. defaults to true do perform statements recovery. \n\n ImportRewrite fImportRewrite concode_elem_sep ImportRemover fImportRemover concode_elem_sep WorkingCopyOwner fOwner concode_elem_sep ASTRewrite fRewrite concode_elem_sep boolean fStatementsRecovery concode_elem_sep List<TextEditGroup> fTextEditGroups concode_elem_sep ICompilationUnit fCu concode_elem_sep IDocument fRememberContent concode_elem_sep CompilationUnit fRoot concode_elem_sep boolean fResolveBindings concode_elem_sep boolean fBindin

'\n                public static void setStatementsRecovery(boolean value) {\n                    // TODO: Implement\n                }\n'

In [75]:
dct[1]['code']

'void function ( boolean arg0 ) { fStatementsRecovery = arg0 ; }'

In [23]:
dct[0]['code']

'int function ( ) { _total = extractList ( ) . size ( ) ; return _total ; }'

In [11]:
import math


def is_prime(number):  # Checks if a given integer is a prime number.
    for i in range (2, int(math.ceil(math.sqrt(number)))+1):
        if number % i == 0:
            return False
    return True

In [12]:
is_prime(23)

True